In [1]:
from bs4 import BeautifulSoup as soup
import requests
import http.cookiejar, urllib.request
import json
from altmetric import Altmetric
import pandas as pd
import time
from collections import Counter
import string
import numpy as np

In [2]:
import nltk
import gensim
import gensim.models.word2vec as w2v
from gensim import corpora
import sklearn.manifold
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
import re
import multiprocessing

In [27]:
#important urls

mainurl = 'http://ascopubs.org'
myurl = ''
#http://ascopubs.org/action/doSearch?ConceptID=33&target=topic&SeriesKey=jco&pageSize=100

#ASCO colorectal cancer ConceptID=145 pageSize=100 startPage=1total results 345
topicid = [145,25] #colorectal and anal cancers, breast cancer, 
topicASCO = ['Colorectal cancer','Breast cancer']
articlenum = [345,1015] #number of articles by topic

#ascourl = 'http://ascopubs.org/action/doSearch?ConceptID=' + str(topicid) + '&target=topic&SeriesKey=jco&pageSize=100&startPage='
#http://ascopubs.org/action/doSearch?ConceptID=145&target=topic&SeriesKey=jco&pageSize=100&startPage=2

thelanceturl = 'http://www.thelancet.com/journals/lanonc/issues'
thelancet = 'http://www.thelancet.com'

natureurl = 'https://www.nature.com/nrclinonc/archive/index.html'
nature = 'http://www.nature.com'

caurl = 'http://onlinelibrary.wiley.com/journal/10.3322/(ISSN)1542-4863/issues'

cellurl = 'http://www.cell.com/cancer-cell/archive'
cell = 'http://www.cell.com'
#http://ascopubs.org/action/doSearch?ConceptID=33&target=topic&SeriesKey=jco
#http://ascopubs.org/action/doSearch?ConceptID=33&target=topic&SeriesKey=jco&pageSize=100
#http://ascopubs.org/action/doSearch?ConceptID=33&target=topic&SeriesKey=jco&pageSize=100&startPage=1
#http://ascopubs.org/action/doSearch?ConceptID=33&target=topic&SeriesKey=jco&pageSize=100&startPage=2
alturl = 'https://api.altmetric.com/v1'

In [28]:
def findAscoContainers(url,page):
    cj = http.cookiejar.CookieJar()
    opener = urllib.request.build_opener(urllib.request.HTTPCookieProcessor(cj))
    r = opener.open(url + str(page))
    #open the connection grab the page
    pagehtml = r.read()
    #html parsing
    pagesoup = soup(pagehtml, "html.parser")
    containers = pagesoup.findAll("article",{"class":"searchResultItem"})
    return containers

In [29]:
def getTopic(title):
    topic = []
    titlel = []
    for i in range(0,len(title)):
        titlel.append(0)
        titlel[i] = title[i].lower()
        topic.append(0)
        if 'lung' in titlel[i]: 
            topic[i] = 'Lung cancer'
        elif 'nsclc' in titlel[i]: 
            topic[i] = 'Breast cancer'
        elif 'breast' in titlel[i]: 
            topic[i] = 'Breast cancer'
        elif 'colon' in titlel[i]: 
            topic[i] = 'Colon cancer'
        elif 'crc' in titlel[i]: 
            topic[i] = 'Colon cancer'
        elif 'melanoma' in titlel[i]: 
            topic[i] = 'Melanoma'
        elif 'pancreas' in titlel[i]: 
            topic[i] = 'Pancreatic cancer'
        elif 'lymphoma' in titlel[i]: 
            topic[i] = 'Lymphoma'
        elif 'leukemia' in titlel[i]: 
            topic[i] = 'Leukemia'
        elif 'gastric' in titlel[i]: 
            topic[i] = 'Gastric cancer'
        else:
            topic[i] = 'Cancer'
    return topic

In [30]:
def searchAPI(href, topic):
    a = Altmetric()
    title = []
    doi = []
    abstract = []
    journal = []
    authors = []
    citedby = []
    tweets = []
    altscore = []
    genecount = np.zeros((len(href), 5))
    allabstracts = ""
    for i in range(0,len(href)):
        new = []
        #try:
        new = a.doi(href[i])
        title.append(i) 
        doi.append(i) 
        abstract.append(i) 
        journal.append(i) 
        authors.append(i) 
        citedby.append(i)
        tweets.append(i)
        altscore.append(i)
        doi[i] = href[i]
        try: 
            title[i] = new['title']
        except:
            title[i] = 'blank'
        try: 
            abstract[i] = new['abstract']
        except:
            abstract[i] = 'blank'
        try: 
            journal[i] = new['journal']
        except:
            journal[i] = 'blank'
        try: 
            authors[i] = new['authors']
        except:
            authors[i] = 'blank'
        try: 
            citedby[i] = new['cited_by_posts_count']
        except:        
            citedby[i] = 0
        try: 
            tweets[i] = new['cited_by_tweeters_count']
        except:         
            tweets[i] = 0
        try: 
            altscore[i] = new['score']
        except:         
            altscore[i] = 0

        allabstracts += str(abstract[i])
        word = Counter(abstract[i].translate(string.punctuation).lower().split())
        genecount[i][0] = word['kras']
        genecount[i][1] = word['braf']
        genecount[i][2] = word['her2']
        genecount[i][3] = word['pik3ca']
        genecount[i][4] = word['ntrk']
        time.sleep(1)
    if topic == True:
        topic = getTopic(title)
    else:
        topic = []    
    return title, doi, abstract, journal, authors, citedby, tweets, altscore, genecount, allabstracts, topic

In [31]:
df = pd.DataFrame(data=None, columns = ['title', 'doi', 'journal', 'authors', 'citedby', 'tweets', 'altscore', 'topic'])
df3 = pd.DataFrame(data=None, columns = ['title', 'doi', 'abstract', 'topic'])

In [32]:
#made for asco


genes = "KRAS BRAF HER2 TP53 PIK3CA NTRK"
allabstracts = ""
j=0
#for each topic
for k in range(0,len(topicid)):
#search through the pages of journal
    j=0
    while (j*100 < articlenum[k] -1):
        #open webpage and enable cookies
        ascourl = 'http://ascopubs.org/action/doSearch?ConceptID=' + str(topicid[k]) + '&target=topic&SeriesKey=jco&pageSize=100&startPage='
        containers = findAscoContainers(ascourl, j)
        d=[]
        df2 = []
        href = []
        for i in range(0,len(containers)):
            href.append(containers[i].div.a["href"])
            href[i] = href[i].replace('/doi/full/','')
        [title,doi,abstract,journal,authors,citedby,tweets,altscore,genecount,allabs,new] = searchAPI(href, False)
        allabstracts += allabs   
        
        d = []
        d3 = []
        df2 = []
        df4 = []
        d = {'title': title, 'doi': doi, 'journal': journal, 'authors': authors, 'citedby': citedby, 'tweets': tweets, 'altscore': altscore, 'topic': topicASCO[k]}
        d3 = {'title': title, 'doi': doi, 'abstract': abstract, 'topic': topicASCO[k]}
        df2 = pd.DataFrame(data=d)
        df = df.append(df2, ignore_index=True)
        df4 = pd.DataFrame(data=d3)
        df3 = df3.append(df4, ignore_index=True)
        print("Reading article..." + str(j*100) + " on " + str(topicASCO[k]))
        j=j+1

Reading article...0 on Colorectal cancer
Reading article...100 on Colorectal cancer
Reading article...200 on Colorectal cancer
Reading article...300 on Colorectal cancer
Reading article...0 on Breast cancer
Reading article...100 on Breast cancer
Reading article...200 on Breast cancer
Reading article...300 on Breast cancer
Reading article...400 on Breast cancer
Reading article...500 on Breast cancer
Reading article...600 on Breast cancer
Reading article...700 on Breast cancer
Reading article...800 on Breast cancer
Reading article...900 on Breast cancer
Reading article...1000 on Breast cancer


In [33]:
df

,altscore,authors,citedby,doi,journal,title,topic,tweets
0,20.00,"[Vivian Colón-López, Meredith S. Shiels, Mark ...",5,10.1200/JCO.2017.74.9291,Journal of Clinical Oncology,Anal Cancer Risk Among People With HIV Infecti...,Colorectal cancer,3
1,20.50,"[Jianming Xu, Tae Won Kim, Lin Shen, Virote Sr...",22,10.1200/JCO.2017.74.3245,Journal of Clinical Oncology,"Results of a Randomized, Double-Blind, Placebo...",Colorectal cancer,20
2,30.40,"[Charles S. Fuchs, Donna Niedzwiecki, Harvey J...",28,10.1200/JCO.2017.74.2130,Journal of Clinical Oncology,"Adjuvant Chemoradiotherapy With Epirubicin, Ci...",Colorectal cancer,25
3,165.95,"[Xinwei Hua, Amanda I. Phipps, Andrea N. Burne...",64,10.1200/JCO.2017.72.3569,Journal of Clinical Oncology,Timing of Aspirin and Other Nonsteroidal Anti-...,Colorectal cancer,39
4,47.95,"[Jeremy C. Jones, Lindsay A. Renfro, Humaid O....",70,10.1200/JCO.2016.71.4394,Journal of Clinical Oncology,Non-V600 BRAF Mutations Define a Clinically Di...,Colorectal cancer,55
5,22.30,"[Naruhiko Ikoma, Y. Nancy You, Brian K. Bednar...",17,10.1200/JCO.2016.72.1464,Journal of Clinical Oncology,Impact of Recurrence and Salvage Surgery on Su...,Colorectal cancer,14
6,14.95,"[Eric Van Cutsem, Jeroen Dekervel]",13,10.1200/JCO.2017.72.7057,Journal of Clinical Oncology,Not All BRAF-Mutant Metastatic Colorectal Canc...,Colorectal cancer,12
7,18.95,"[Fay Kastrinos, Hajime Uno, Chinedu Ukaegbu, C...",26,10.1200/JCO.2016.69.6120,Journal of Clinical Oncology,Development and Validation of the PREMM5 Model...,Colorectal cancer,18
8,54.15,"[Lindsay A. Renfro, Richard M. Goldberg, Axel ...",40,10.1200/JCO.2016.71.5771,Journal of Clinical Oncology,Clinical Calculator for Early Mortality in Met...,Colorectal cancer,30
9,34.70,"[Bas Groot Koerkamp, Eran Sadot, Nancy E. Keme...",22,10.1200/JCO.2016.71.8346,Journal of Clinical Oncology,Perioperative Hepatic Arterial Infusion Pump C...,Colorectal cancer,16


In [34]:
#The Lancet
cj = http.cookiejar.CookieJar()
opener = urllib.request.build_opener(urllib.request.HTTPCookieProcessor(cj))
r = opener.open(thelanceturl)
pagehtml = r.read()

#html parsing
pagesoup = soup(pagehtml, "html.parser")

#grab hyperlinc for journals
containers = pagesoup.findAll("a",{"class":"issueLinkCon"})
href = []
for i in range(0,len(containers)):
    href.append(i)
    href[i] = containers[i]['href']
    

for k in range(0,len(href)):
    cj = http.cookiejar.CookieJar()
    opener = urllib.request.build_opener(urllib.request.HTTPCookieProcessor(cj))
    r = opener.open(thelancet+href[k])
    pagehtml = r.read()
    
    #html parsing
    pagesoup = soup(pagehtml, "html.parser")
    
    containers = []
    children = []
    
    #grab relevant html classes
    containers = pagesoup.find("div",{"class":"articleCitations"})
    children = containers.findChildren()
    
    #find where articles are located
    start = 0
    stop = len(children)
    for i in range(0,len(children)):
        if children[i].string == 'Articles':
            start = i
        if children[i].string == 'Review':
            stop = i
    newchildren = children[start:stop]     
    
    #sort out only doi
    child = []
    for i in range(0,len(newchildren)):
        child.append(i)
        child[i] = newchildren[i].find("div",{"class":"doi"})
        
    #remove None and duplicates
    child = list(filter(None, child))
    child = list(set(child))
    
    doi = []
    for i in range(0,len(child)):
        doi.append(i)
        doi[i] = child[i].string
        doi[i] = doi[i].replace('DOI: http://dx.doi.org/','')
        
    [title,doi,abstract,journal,authors,citedby,tweets,altscore,genecount,allabs,topic] = searchAPI(doi, True)
    allabstracts += allabs   
    
    d = []
    d3 = []
    df2 = []
    df4 = []
    d = {'title': title, 'doi': doi, 'journal': journal, 'authors': authors, 'citedby': citedby, 'tweets': tweets, 'altscore': altscore, 'topic': topic}
    d3 = {'title': title, 'doi': doi, 'abstract': abstract, 'topic': topic}
    df2 = pd.DataFrame(data=d)
    df = df.append(df2, ignore_index=True)
    df4 = pd.DataFrame(data=d3)
    df3 = df3.append(df4, ignore_index=True)

In [35]:
#Nature
cj = http.cookiejar.CookieJar()
opener = urllib.request.build_opener(urllib.request.HTTPCookieProcessor(cj))
r = opener.open(natureurl)
pagehtml = r.read()

#html parsing
pagesoup = soup(pagehtml, "html.parser")

containers = pagesoup.findAll("p",{"class":"issue"})
href = []
for i in range(0,len(containers)):
    href.append(i)
    href[i] = containers[i].a['href']
    
for k in range(0,len(href)):
    cj = http.cookiejar.CookieJar()
    opener = urllib.request.build_opener(urllib.request.HTTPCookieProcessor(cj))
    r = opener.open(nature+href[k])
    pagehtml = r.read()
    
    #html parsing
    pagesoup = soup(pagehtml, "html.parser")
    
    containers = []
    children = []
    
    #grab relevant html classes
    containers = pagesoup.findAll("div",{"class":"article"})
    
    doi = []
    for i in range(0,len(containers)):
        doi.append(i)
        doi[i] = str(containers[i].find("span",{"class":"doi"}))
        doi[i] = doi[i].replace('<span class="doi"><abbr title="Digital Object Identifier">doi</abbr>:','')
        doi[i] = doi[i].replace('</span>','')
        
    [title,doi,abstract,journal,authors,citedby,tweets,altscore,genecount,allabs,topic] = searchAPI(doi, True)
    allabstracts += allabs   

    d = []
    d3 = []
    df2 = []
    df4 = []
    d = {'title': title, 'doi': doi, 'journal': journal, 'authors': authors, 'citedby': citedby, 'tweets': tweets, 'altscore': altscore, 'topic': topic}
    d3 = {'title': title, 'doi': doi, 'abstract': abstract, 'topic': topic}    
    df2 = pd.DataFrame(data=d)
    df = df.append(df2, ignore_index=True)
    df4 = pd.DataFrame(data=d3)
    df3 = df3.append(df4, ignore_index=True)

In [36]:
#Cancer cell
cj = http.cookiejar.CookieJar()
opener = urllib.request.build_opener(urllib.request.HTTPCookieProcessor(cj))
r = opener.open(cellurl)
pagehtml = r.read()

#html parsing
pagesoup = soup(pagehtml, "html.parser")

containers = pagesoup.findAll("a",{"class":"issueLinkCon"})

href = []
for i in range(0,len(containers)):
    href.append(i)
    href[i] = containers[i]['href']
    
for k in range(0,len(href)):
    cj = http.cookiejar.CookieJar()
    opener = urllib.request.build_opener(urllib.request.HTTPCookieProcessor(cj))
    r = opener.open(cell+href[k])
    pagehtml = r.read()
    
    #html parsing
    pagesoup = soup(pagehtml, "html.parser")
    
    containers = []
    children = []
    
    #grab relevant html classes
    containers = pagesoup.findAll("div",{"class":"articleTitle"})
    
    doi = []
    for i in range(0,len(containers)):
        
        newref = containers[i].a['href']
        
        cj = http.cookiejar.CookieJar()
        opener = urllib.request.build_opener(urllib.request.HTTPCookieProcessor(cj))
        r = opener.open(cell+newref)
        pagehtml = r.read()
        
        #html parsing
        pagesoup = soup(pagehtml, "html.parser")
        
        containersDOI = pagesoup.find("div",{"class":"doi"})
        
        containersDOI.a['href']
        
        doi.append(i)
        doi[i] = containersDOI.a['href']
        doi[i] = doi[i].replace('http://dx.doi.org/','')
        
        [title,doi,abstract,journal,authors,citedby,tweets,altscore,genecount,allabs,topic] = searchAPI(doi, True)
        allabstracts += allabs   

    d = []
    d3 = []
    df2 = []
    df4 = []
    d = {'title': title, 'doi': doi, 'journal': journal, 'authors': authors, 'citedby': citedby, 'tweets': tweets, 'altscore': altscore, 'topic': topic}
    d3 = {'title': title, 'doi': doi, 'abstract': abstract, 'topic': topic}
    df2 = pd.DataFrame(data=d)
    df = df.append(df2, ignore_index=True)
    df4 = pd.DataFrame(data=d3)
    df3 = df3.append(df4, ignore_index=True)

ConnectionError: HTTPConnectionPool(host='api.altmetric.com', port=80): Max retries exceeded with url: /v1/doi/10.1016/j.ccr.2013.04.003 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f95b06f3550>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution',))

In [40]:
df.to_csv('dataFrameMain.csv')

In [41]:
df3.to_csv('dataFrameAbstract.csv')

In [ ]:
df4

In [ ]:
df4

In [37]:
df3

,abstract,doi,title,topic
0,Purpose People with HIV infection have an elev...,10.1200/JCO.2017.74.9291,Anal Cancer Risk Among People With HIV Infecti...,Colorectal cancer
1,Purpose Trifluridine/tipiracil (TAS-102) was e...,10.1200/JCO.2017.74.3245,"Results of a Randomized, Double-Blind, Placebo...",Colorectal cancer
2,Purpose After curative resection of gastric or...,10.1200/JCO.2017.74.2130,"Adjuvant Chemoradiotherapy With Epirubicin, Ci...",Colorectal cancer
3,Purpose Regular use of aspirin is associated w...,10.1200/JCO.2017.72.3569,Timing of Aspirin and Other Nonsteroidal Anti-...,Colorectal cancer
4,Purpose Molecular diagnostic testing has becom...,10.1200/JCO.2016.71.4394,Non-V600 BRAF Mutations Define a Clinically Di...,Colorectal cancer
5,Purpose After preoperative chemoradiotherapy f...,10.1200/JCO.2016.72.1464,Impact of Recurrence and Salvage Surgery on Su...,Colorectal cancer
6,blank,10.1200/JCO.2017.72.7057,Not All BRAF-Mutant Metastatic Colorectal Canc...,Colorectal cancer
7,Purpose Current Lynch syndrome (LS) prediction...,10.1200/JCO.2016.69.6120,Development and Validation of the PREMM5 Model...,Colorectal cancer
8,Purpose Factors contributing to early mortalit...,10.1200/JCO.2016.71.5771,Clinical Calculator for Early Mortality in Met...,Colorectal cancer
9,Purpose To investigate whether perioperative h...,10.1200/JCO.2016.71.8346,Perioperative Hepatic Arterial Infusion Pump C...,Colorectal cancer


In [ ]:
i

In [38]:
k

56

In [ ]:
df3

In [39]:
len(href)

96

In [5]:
from clinical_trials import Trials


ImportError: cannot import name 'Trials'